# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Основными этапами нашего проекта станут:  
* Открытие файла с данными и изучение общей информации
* Предобработка данных
* Расчеты и добавление необходимых для анализа результатов
* Анализ данных
* Проверка гипотез
* Формулирование основных выводов

Проект выполнен в **Jupyter Notebook**, версия **Python** 3.7.8.  
В проекте использованы библиотеки **Pandas**, **PyMyStem3**, а также модуль **IPython**. 

## Открытие файла с данными и изучение общей информации

In [1]:
# Импортируем необходимые библиотеки.
import pandas as pd
from IPython.display import display
from pymystem3 import Mystem

# Прочитаем датасет.
data = pd.read_csv('/datasets/data.csv')
data.info()
print()
print('\033[1m' + 'Таблица № 1. Первые десять строк исходного датасета')
display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

Таблица № 1. Первые десять строк исходного датасета


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Исходный датасет состоит из 21525 строк и 12 столбцов.  
* 2 столбца содержат данные формата `float64` (вещественные числа) 
* 5 столбцов содержат данные формата `int64` (целые числа)
* 5 столбцов содержат данные формата `object`

Предварительно проанализировав данные, был построен план по предобработке данных:
1. Стало ясно, что в столбцах **days_employed** и **total_income** имеются пропуски (по 2174 в каждом), которые необходимо обработать.
2. Стоит отметить, что в столбце **days_employed** хранятся данные в формате `float64`. В данном столбце, исходя из его названия, хранятся данные о количестве дней, в течение которых клиент банка был трудоустроен на момент создания данной таблицы. Очевидно, что количество дней - это величина, которую предпочтительнее отображать целыми числами (формат `int64`). Есть предположение, что скорее всего количество дней трудоустройства рассчитываются по формуле, содержащей в себе оператор деления, в результате чего в столбце появляются вещественные числа. Целесообразно приведение формата данных чисел к формату `int64`.
3. В столбцах **children** и **days_employed** имеются отрицательные значения. Такие значения не имеют смысла (вероятно была допущена ошибка при сборе данных). Их необходимо преобразовать в положительные значения.
4. В столбце **days_employed** имеются аномальные значения.Так, например, у категории клиентов "пенсионер" количество дней стажа превышает максимально возможное.
5. В столбце **dob_years** имеются нулевые значения. Необходимо заменить их на средния значения для каждой категории клиентов (категория по столбцу **income_type**).
6. В столбцах **education** и **family_status** необходимо добиться единообразия в размере букв.
7. В столбцах **family_status**, **income_type**, purpose необходимо провести проверку на дубликаты.
8. В столбце **total_income** имеет смысл отображать числа в формате `int64`.

## Предобработка данных

### Обработка пропусков

In [2]:
# Избавимся от отрицательных значений.
data['days_employed'] = data['days_employed'].abs()
data['children'] = data['children'].abs()

# Создается функция для перевода значения дней, 
# выраженного предположительно в часах, в дни.
def hours_to_days(presumably_hours):
    # Исходим из того, что человек за жизнь может
    # официально трудиться 22265 часов или 61 год.
    if presumably_hours > 22265:
        return presumably_hours / 24
    return presumably_hours

# Применим функцию.
data['days_employed'] = data['days_employed'].apply(hours_to_days)

# Создается список с категориями клиентов. 
#Это неоднократно пригодится далее для замены NaN-значений.
income_types = data.groupby('income_type').sum().index

# Создается цикл для замены нулевых значений в столбце dob_years 
# на средние значения в каждой категории клиентов.
for income_type in income_types:
    data_dob_mean = data.groupby('income_type')['dob_years'].mean()
    data.loc[
        data['income_type'] == income_type, 'dob_years'] = data.loc[
        data['income_type'] == income_type, 'dob_years'].replace(
        0, data_dob_mean[income_type]
    )

# После замены нулевых значений в столбце dob_years изменился тип данных. 
# Возвращаем тип данных int.
data['dob_years'] = data['dob_years'].astype(int)

In [3]:
# Создается функция для исключения аномалий в столбце days_employed,
# связанных с невозможными значениями количества дней стажа
def reduction(row):
    days_employed = row['days_employed']
    dob_years = row['dob_years']
    calc_days = (dob_years-14) * 365
    if  days_employed > calc_days:
        return calc_days
    return days_employed

# Применяем функцию.
data['days_employed'] = data.apply(reduction, axis=1)

# Обработка данных для замены NaN-значений завершена. Создается цикл
# для замены NaN-значений в столбцах income_type и total_income.
for income_type in income_types:
    data_mean = data.groupby('income_type')['days_employed'].mean()
    data_median = data.loc[
        data['income_type'] == income_type, 'total_income'].median()
    data.loc[
        data['income_type'] == income_type, 'days_employed'] = data.loc[
        data['income_type'] == income_type, 'days_employed'].fillna(
        data_mean[income_type]
    )
    data.loc[
        data['income_type'] == income_type, 'total_income'] = data.loc[
        data['income_type'] == income_type, 'total_income'].fillna(
        data_median
    )
print('\033[1m' + 'Таблица № 2. Количество пропусков в стольбцах датасета')    
display(data.isna().sum())

Таблица № 2. Количество пропусков в стольбцах датасета


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

В первую очередь было необходимо обработать пропуски типа `NaN` в столбцах **days_employed** и **total_income**.  
Всего пропусков 2174 в каждом столбце или 10 % от общего числа данных в этих столбцах.  
10 % — это существенная доля, поэтому в целях сохранения объективности выводов исследования было принято решение не удалять эти данные.
Значения типа `NaN` присутствуют в столбцах **days_employed** и **total_income** у одних и тех же клиентов. Причиной, по которым, вероятно, эти данные пропущены, является остутствие у банка трудовых книжек или форм 2-НДФЛ данных клиентов.

Чтобы понять на какие значения нужно заменить пропуски в столбце, необходимо привести все аномалии в репрезентативный вид. 
1. Методом `abs()` отрицательные значения были преобразованы в положительные.
2. Обнаружены невозможно большие значения дней стажа у некоторых клиентов. Предположительно в столбце **days_employed** у данной категории клиентов указано количество часов, в течение которых они были трудоустроены, а не дней. В этой связи было принято решение перевести невозможно большие значения часов трудоустройства в дни. Для этого был выбран порог значения количества дней, после которого значение принимается невозможно большим — 22265 дней. Данное значение рассчитано исходя из того, что в Российской Федерации человек можеть начать свой трудовой стаж в 14 лет. Максимальный возраст в данной таблице 75 лет. (75-14) * 365 = 22265 дней. Таково максимально возможное количество дней трудоустройства для клиентов в этом датасете. Для перевода аномальных значений из часов в дни используется функция **hours_to_days**.
3. В ходе рассмотрения столбца **dob_years** выявлено, что имеются нулевые значения. При помощи цикла нулевые значения заменены на средния значения для каждой категории клиентов.
3. Путем метода `idxmax()` выявлено, что в столбце **days_employed** в строке 16335 хранится значение 18388, 9 дней трудоустройства (50,3 года). При этом возраст клиента 61 год. Начало трудового стажа в 11 лет невозможно. Создана формула **reduction**, определяющая подобные аномалии и заменяющая количество дней трудоустройства в аномалиях на максимально теоретическое расчетное количество (разность возраста из столбца **dob_years** и 14 лет, умноженная на 365 дней).
4. Теперь, когда все аномалии исправлены, можно начать расчет значений, которыми потом можно будет заполнить пропуски. Предлагается посчитать среднее значение дней стажа для каждой категории клиентов (столбец **income_type**) и заменить ими пропуски в столбце **days_employed**. После замены аномалий замена данных в этом столбце на среднее значение каждой категории клиентов уместно. Далее предлагается посчитать медианные значения суммарного дохода для каждой категории граждан и заменить ими пропуски в столбце **total_income**. Медианное значение устойчиво к "выбросам" возможным в данном столбце.

### Замена типа данных

In [4]:
# Столбцам days_employed и total_income придается формат int64.
data = data.astype({'days_employed': 'int', 'total_income': 'int'})

print('\033[1m' + 'Таблица № 3. Столбцы days_employed и total_income' +
     ' после изменения типа данных') 
display(data[['days_employed', 'total_income']])

Таблица № 3. Столбцы days_employed и total_income после изменения типа данных


,days_employed,total_income
0,8437,253875
1,4024,112080
2,5623,145885
3,4124,267628
4,14177,158616
...,...,...
21520,4529,224791
21521,14330,155999
21522,2113,89672
21523,3112,244093


**Вывод**

Значения в столбцах **days_employed** и **total_income** хранятся в формате `float64`. Целесообразнее хранить их в формате `int64` (отображать количество дней и денег с 5 знаками после запятой нецелесообразно).  
Наиболее оптимальным методом перевода значений в другой тип данных в данном случае является `astype()`.  
В результате применения данного метода значения в вышеуказанных столбцах приняли формат `int64`.

### Обработка дубликатов

In [5]:
# Проверим датасет на наличие полных дубликатов.
print('Количество полных дубликатов в датасете составляет',
     data.duplicated().sum())

# Удалим полные дубликаты.
data = data.drop_duplicates()

print('Количество полных дубликатов после удаления:',
     data.duplicated().sum())

# Дубликаты могут теоритически находиться в столбцах 
# education, education_id, family_status,
# family_status_id, gender, income_type, debt. 
# Проверим это методом value_counts().

columns = ['education','education_id','family_status', 
           'family_status_id', 'gender', 'income_type', 'debt']

for n, column in zip(range(4,11), columns):
    print('\033[1m' + f'Таблица № {n}. Проверка столбца {column}' +
         ' на дубликаты методом value_counts()') 
    display(data[column].value_counts())
    
# Анализ показал, что дубликаты имеются в столбце education.
# Также в столбце gender есть строка со значением XNA. 
# Отнесем её к мужскому полу, как к наименее представленному в выборке.
data['gender'] = data['gender'].replace('XNA', 'M')

# В столбце education строки приведены к нижнему регистру
data['education'] = data['education'].str.lower()

Количество полных дубликатов в датасете составляет 54
Количество полных дубликатов после удаления: 0
Таблица № 4. Проверка столбца education на дубликаты методом value_counts()


среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Таблица № 5. Проверка столбца education_id на дубликаты методом value_counts()


1    15188
0     5251
2      744
3      282
4        6
Name: education_id, dtype: int64

Таблица № 6. Проверка столбца family_status на дубликаты методом value_counts()


...

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

Таблица № 9. Проверка столбца income_type на дубликаты методом value_counts()


сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Таблица № 10. Проверка столбца debt на дубликаты методом value_counts()


0    19730
1     1741
Name: debt, dtype: int64

**Вывод**

Проведена обработка дубликатов. Удалены полные дубликаты.  
Теоретически появление дубликатов возможно в столбцах:  
* education 
* education_id
* family_status
* family_status_id
* gender
* income_type
* debt 

Методом `value_counts()` определен столбец, имеющий дубликаты. Им оказался **education**. В столбце **gender** определено значение `XNA` не имеющее смысла для исследования.  
Принято решение отнести его к мужскому полу, как к наименее представленному в выборке (при помощи метода `replace()`).  
Вероятной причиной появления дубликтов в столбце **education** является отсутствие в банке правил внесения сотрудниками банка исходной информации о его клиентах или нарушение таковых. В данном столбце строки приведены к нижнему регистру при помощи метода `srt.lower()`. В результате применения метода в столбце **education** дубликатов больше нет.

### Лемматизация

In [6]:
purposes = data['purpose']
# Изучая информацию, полученную методом value_counts, 
# уже можно на глаз определить будущие категории.

# Для последующей категоризации целей кредита 
# проводится лемматизация в столбце purpose
# Результаты вносятся в список lemmas. Используется библиотека PyMyStem.
m = Mystem()   
lemmas = []
for row in purposes:
    lemma = m.lemmatize(row)
    lemmas.append(lemma)

lemmas

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['покупка', ' ', 'жилье', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['недвижимость', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'п

**Вывод**

Для последующей категоризации целей кредита проведена лемматизация в столбце **purpose** при помощи библиотеки **PyMyStem**. Результаты лемматизиации занесены в список **lemmas**. Все цели кредита можно свести к пяти леммам (категориям):  
* жилье
* автомобиль
* свадьба
* образование
* недвижимость 

Это и будет словарем категорий.

### Категоризация данных

Создадим список с категориями, соответствующими каждой лемме.  
0 - жильё  
1 - автомобиль  
2 - свадьба  
3 - образование  
4 - недвижимость

In [7]:
# Создается список с категориями, соответствующими каждой лемме.
categories = []
for string in lemmas:
    if 'жилье' in string:
        categories.append(0)
    if 'автомобиль' in string:
        categories.append(1)
    if 'свадьба' in string:
        categories.append(2)
    if 'образование' in string:
        categories.append(3)
    if 'недвижимость'in string:
        categories.append(4)
# Список добавляется в датасет.       
data['purpose_category'] = categories

print('\033[1m' + 'Таблица № 11. Датасет после добавления столбца ' +
     'purpose_category') 
display(data.head(10))

Таблица № 11. Датасет после добавления столбца purpose_category


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,2
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,0
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,0
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,2
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,0


**Вывод**

Создан список категорий **categories**, соответствующими каждой лемме. Данный список добавляется в таблицу **data**, следующим после столбца **purpose**.

## Проверка гипотез

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
# Заменим количество детей с 20 на 5 (20 детей в семье - аномалия).
data['children'] = data['children'].replace(20, 5)
# Поличим долю клиентов, не вернувших долг вовремя 
# от общего количества клиентов в разбивке по количеству детей.
# Создадим функцию.
def get_ratio_debt(dataset, column_name):
    ratio = dataset.groupby(column_name)['debt'].mean()
    return ratio
    
part_by_children = get_ratio_debt(data, 'children')
display(part_by_children)

children
0    0.075353
1    0.091639
2    0.094542
3    0.081818
4    0.097561
5    0.094118
Name: debt, dtype: float64

**Вывод**

Стоит обратить внимание, что во время построения данной таблицы была обнаружена аномалия в столбце children.  
У 76 клиентов 20 детей в семье.  
Принято решение отнести эти случаи к категории "5 детей в семье", как к ближайшей по "размеру" категории.
Расчет доли клиентов, не вернувших долг вовремя от общего количества клиентов в разбивке по количеству детей в семье,  подтверждает наличие зависимости между наличием детей и возвратом кредита в срок среди указанных клиентов.  
Клиенты без детей чаще возвращают кредит вовремя, чем клиенты с детьми на 1,5 %. Рассмотрев клиентов, у которых есть дети, чёткого вывода о наличии зависимости между количеством детей и возвратом кредита в срок составить не удалось. Примерно в 9 % случаев клиенты с детьми не возвращают кредит вовремя.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
# Посчитем долю клиентов, не вернувших кредит в срок, 
# в разбивке по семейному положению.
# Для наглядности применен метод сортировки sort_values().
part_by_family_status = get_ratio_debt(data, 'family_status')
display(part_by_family_status.sort_values(ascending=False))

family_status
Не женат / не замужем    0.097509
гражданский брак         0.093202
женат / замужем          0.075421
в разводе                0.071130
вдовец / вдова           0.065693
Name: debt, dtype: float64

**Вывод**

Расчет доли клиентов, не вернувших кредит вовремя от общего количества клиентов в разбивке по семейному положению,  подтверждает наличие зависимости между семейным положением и возвратом кредита в срок среди указанных клиентов.  
Официально не оформившие свои отношения клиенты чаще не возвращают вовремя кредит (9-10 %), чем клиенты состоящие в браке, или имевшие опыт брачной жизни (7-8 %). 
Стоит отметить, что при составлении типов семейного положения банком было неверно трактовано понятие гражданского брака.  
Категории "гражданский брак" и "женат / замужем" - это одно и то же. Банком подразумевается, что гражданский брак - это сожительство людей, состоящих в отношениях, без официальной регистрации отношений в органах власти. Так и стоило назвать эту категорию - "сожительство". 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
# Для выявления зависимости необходимо разделить доходы клиентов
# на несколько групп и провести категоризацию. Создадим функцию:
def income_group(total_income):
    if total_income <= 36000:
        return 'менее среднего заработка'
    if total_income <= 97000:
        return 'выше среднего заработка'
    if total_income <= 136000:
        return 'средний класс'
    return 'выше среднего класса'

data['income_group'] = data['total_income'].apply(income_group)

# Посчитаем долю клиентов, не вернувших кредит в срок, 
# в разбивке по категориям доходов.
# Для наглядности применен метод сортировки sort_values()
part_by_income_group = get_ratio_debt(data, 'income_group')
display(part_by_income_group.sort_values(ascending=False))

income_group
средний класс               0.084161
выше среднего заработка     0.080585
выше среднего класса        0.080196
менее среднего заработка    0.053333
Name: debt, dtype: float64

**Вывод**

Для корректного определения наличия зависимости между уровнем дохода и возвратом кредита в срок проведена категоризация доходов клиентов.  
За основу взят среднемесячный доход на душу населения РФ в 2020 г. по данным Росстата (примерно 36000 руб.).  
Далее умозрительно были созданы категории "выше среднего заработка", "средний класс" и "выше среднего класса". Стоит отметить, что понятие среднего класса имеет множество разных толкований, и, конечно же, не имеет точной границы в размере среднемесячного дохода человека.  
Расчет доли клиентов, не вернувших долг вовремя от общего количества клиентов в разбивке по категориям дохода,  подтверждает наличие зависимости между уровнем дохода и возвратом кредита в срок среди указанных клиентов.  
Так, представители категорий "средний класс", "выше среднего класса", "выше среднего заработка" возращают кредит с нарушением сроков примерно на 3 % чаще, чем представители категории "менее среднего заработка".

- Как разные цели кредита влияют на его возврат в срок?

Напомним категории:  
0 - жильё  
1 - автомобиль  
2 - свадьба  
3 - образование  
4 - недвижимость

In [11]:
# Посчитаем долю клиентов, не вернувших кредит в срок, 
# в разбивке по категориям целей получения кредита.
# Для наглядности применен метод сортировки sort_values().
part_by_purpose_category = get_ratio_debt(data, 'purpose_category')
display(part_by_purpose_category.sort_values(ascending=False))

purpose_category
1    0.093547
3    0.092177
2    0.079657
4    0.074610
0    0.069043
Name: debt, dtype: float64

**Вывод**

Расчет доли клиентов, не вернувших долг вовремя от общего количества клиентов в разбивке по категориям целей получения кредита,  подтверждает наличие зависимости между целями кредита и возвратом кредита в срок среди указанных клиентов. Так, в категориях "автомобиль" и "образование" клиенты не возвращают вовремя кредит примерно в 9 % случаев. В категориях "свадьба" и "недвижимость" клиенты не возвращают вовремя кредит примерно в 8 % случаев. В категории "жильё" клиенты не возвращают кредит в срок реже всего - в 7 % случаев.

## Общий вывод

Анализ статистики о платежеспособности клиентов позволил сделать 3 утверждения:
1. Клиенты без детей чаще возвращают кредит в срок, чем клиенты с детьми. Количество детей на исполнение кредитных обязательство не влияет.
2. Официально не оформившие свои отношения клиенты чаще нарушают сроки возврата кредита, чем клиенты состоящие в браке, или имевшие опыт брачной жизни ранее.
3. Клиенты, взявшие кредит на жилищные нужды, чаще возвращают его в срок. Реже всего возвращаются в срок кредиты, взятые на нужды, связанные с автомобилем и образованием.